# Design Patterns in python


##  Common patterns covered in this chapter include:
- Structural patterns :(Facade, Adapter, Composite)
- Behavioral patterns (e.g. Chain of Responsibility, Command, State)
- As well as Python-specific patterns (such as the Borg pattern or optimal use of Decorators)

## 1. GitFetcher & Shared State (Borg Pattern):
- ❌ Problem: Ensuring multiple instances share the latest Git tag.
- ⚠️ Issues Before: Each instance had its own independent tag, leading to inconsistencies.
- ✅ Solution: A class variable _current_tag keeps all instances synchronized​


In [25]:
class GitFetcher:
    _current_tag = None  

    def __init__(self, tag):
        self.current_tag = tag  

    @property
    def current_tag(self):
        if self._current_tag is None:
            raise AttributeError("tag was never set")
        return self._current_tag

    @current_tag.setter
    def current_tag(self, new_tag):
        self.__class__._current_tag = new_tag  

    def pull(self):
        print(f"Pulling from {self.current_tag}")
        return self.current_tag

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [26]:
f1 = GitFetcher("v1.0")
f2 = GitFetcher("v2.0")

print(f1.pull()) 
f1.current_tag = "v3.0"
print(f2.pull()) 


Pulling from v2.0
v2.0
Pulling from v3.0
v3.0


## 2. SharedAttribute Descriptor:
- ❌Problem: Encapsulating shared attributes for cleaner design.
- ⚠️Issue Before: Class attributes lacked proper encapsulation and reuse.
- ✅Solution: A descriptor class manages shared state, improving cohesion.

In [27]:
class SharedAttribute:
    def __init__(self, initial_value=None):
        self.value = initial_value
        self._name = None

    def __get__(self, instance, owner):
        if instance is None:
            return self
        if self.value is None:
            raise AttributeError(f"{self._name} was never set")
        return self.value

    def __set__(self, instance, new_value):
        self.value = new_value

    def __set_name__(self, owner, name):
        self._name = name

In [28]:
class GitFetcher:
    current_tag = SharedAttribute()
    current_branch = SharedAttribute()

    def __init__(self, tag, branch=None):
        self.current_tag = tag
        self.current_branch = branch

## 3. Borg Pattern:
- ❌Problem: Ensuring all instances share the same internal state.
- ⚠️Issue Before: Objects had independent state, leading to inconsistency.
- ✅Solution: Shared dictionary (__ dict __) synchronizes attributes across instances.

In [30]:
class BaseFetcher:
    def __init__(self, source):
        self.source = source

class TagFetcher(BaseFetcher):
    _attributes = {}

    def __init__(self, source):
        self.__dict__ = self.__class__._attributes
        super().__init__(source)

    def pull(self):
        logger.info("pulling from tag %s", self.source)
        return f"Tag = {self.source}"


In [32]:
obj1 = Example(10)
obj2 = Example(20)
print(obj1.value) 

20


## 4. TagFetcher & BranchFetcher (Borg Pattern with Separation):
- ❌Problem: Applying Borg Pattern while keeping fetchers separate.
- ✅Solution: Two fetchers for Git tags and branches, ensuring a clean structure.

In [33]:
class BranchFetcher(BaseFetcher):
    _attributes = {}

    def __init__(self, source):
        self.__dict__ = self.__class__._attributes
        super().__init__(source)


## 5. Builder Pattern:
- ❌Problem: Simplifying object creation when multiple components are involved.
- ⚠️Issue Before: Complex initialization scattered across the code.
- ✅Solution: A builder class encapsulates creation logic for better maintainability.

In [35]:
class CarBuilder:
    def __init__(self):
        self.car = Car()

    def add_engine(self, engine):
        self.car.engine = engine
        return self

    def add_wheels(self, wheels):
        self.car.wheels = wheels
        return self

    def build(self):
        return self.car

#### for example:

In [37]:
class Car:
    def __init__(self):
        self.engine = None
        self.wheels = None

class CarBuilder:
    def __init__(self):
        self.car = Car()

    def add_engine(self, engine):
        self.car.engine = engine
        return self

    def add_wheels(self, wheels):
        self.car.wheels = wheels
        return self

    def build(self):
        return self.car

## 6. Adapter Pattern:
- ❌Problem: Adapting incompatible interfaces.
- ⚠️Issue Before: External dependencies had different method signatures.
- ✅Solution: A wrapper class adapts method signatures cleanly.

In [38]:
class ExternalLibrary:
    def search(self, query):
        return f"Results for {query}"

class Adapter:
    def __init__(self, adaptee):
        self.adaptee = adaptee

    def fetch(self, user_id, username):
        query = f"{user_id}:{username}"
        return self.adaptee.search(query)


adapter = Adapter(ExternalLibrary())
print(adapter.fetch(42, "john_doe"))  

Results for 42:john_doe


## 7. Composite Pattern:
- ❌Problem: Treating single objects and groups uniformly.
- ⚠️Issue Before: Handling individual and grouped items separately was complex.
- ✅Solution: A tree-like structure where composite and individual objects share an interface.

In [39]:
class Product:
    def __init__(self, name, price):
        self.name = name
        self.price = price

class ProductBundle:
    def __init__(self, name, discount, *products):
        self.name = name
        self.discount = discount
        self.products = products

    @property
    def price(self):
        total = sum(product.price for product in self.products)
        return total * (1 - self.discount)


p1 = Product("Product 1", 100)
p2 = Product("Product 2", 200)
bundle = ProductBundle("Bundle", 0.1, p1, p2)
print(bundle.price)  

270.0


## 8. Decorator Pattern:
- ❌Problem: Dynamically adding behaviors without modifying the base class.
- ⚠️Issue Before: Subclassing led to class explosion.
- ✅Solution: Wrappers add behavior dynamically.

In [40]:
class DictQuery:
    def __init__(self, **kwargs):
        self._raw_query = kwargs

    def render(self):
        return self._raw_query

class QueryEnhancer:
    def __init__(self, query):
        self.decorated = query

    def render(self):
        return self.decorated.render()

class RemoveEmpty(QueryEnhancer):
    def render(self):
        original = super().render()
        return {k: v for k, v in original.items() if v}

class CaseInsensitive(QueryEnhancer):
    def render(self):
        original = super().render()
        return {k: v.lower() for k, v in original.items()}

query = DictQuery(key="Value", empty="", none=None, upper="UPPERCASE")
decorated = CaseInsensitive(RemoveEmpty(query))
print(decorated.render())  # {'key': 'value', 'upper': 'uppercase'}

{'key': 'value', 'upper': 'uppercase'}


## 9. Facade Pattern:
- ❌Problem: Simplifying complex subsystems.
- ⚠️Issue Before: Clients had to interact with multiple components.
- ✅Solution: A single unified interface hides complexity.

In [41]:
class SubsystemA:
    def operation(self):
        return "SubsystemA: Ready!"

class SubsystemB:
    def operation(self):
        return "SubsystemB: Go!"

class Facade:
    def __init__(self):
        self.system_a = SubsystemA()
        self.system_b = SubsystemB()

    def operation(self):
        return f"{self.system_a.operation()} and {self.system_b.operation()}"

facade = Facade()
print(facade.operation())  # SubsystemA: Ready! and SubsystemB: Go!

SubsystemA: Ready! and SubsystemB: Go!


## 10. Chain of Responsibility:
- ❌Problem: Handling requests with multiple possible processors.
- ⚠️Issue Before: if-else chains made adding new handlers difficult.
- ✅Solution: Requests pass through a chain of handlers.

In [42]:
class Event:
    def __init__(self, successor=None):
        self.successor = successor

    def process(self, logline):
        if self.can_process(logline):
            return self._process(logline)
        if self.successor:
            return self.successor.process(logline)

    def can_process(self, logline):
        return False

    def _process(self, logline):
        return {}

class LoginEvent(Event):
    def can_process(self, logline):
        return "login" in logline

    def _process(self, logline):
        return {"event": "login"}

class LogoutEvent(Event):
    def can_process(self, logline):
        return "logout" in logline

    def _process(self, logline):
        return {"event": "logout"}

chain = LoginEvent(LogoutEvent())
print(chain.process("User login"))  # {'event': 'login'}

{'event': 'login'}


## 11. Template Method Pattern for Data Processing:
- ❌Problem: Duplicate logic in event classes (LoginEvent, LogoutEvent)and hard to extend; relied on if-else conditions.
- ✅Subclasses override only specific parts,Base class (Event) defines a common structure.

In [45]:
class Event:
    def process(self, logline):
        if self.can_process(logline):
            return self._process(logline)
        raise ValueError("Cannot process logline")

    def can_process(self, logline):
        raise NotImplementedError

    def _process(self, logline):
        raise NotImplementedError

class LoginEvent(Event):
    def can_process(self, logline):
        return "login" in logline

    def _process(self, logline):
        return {"event": "login", "logline": logline}

class LogoutEvent(Event):
    def can_process(self, logline):
        return "logout" in logline

    def _process(self, logline):
        return {"event": "logout", "logline": logline}

event = LoginEvent()
result = event.process("User login successfully")
print(result)  # {'event': 'login', 'logline': 'User login successfully'}

{'event': 'login', 'logline': 'User login successfully'}


## 12. Command Pattern (Deferred Execution):
- ❌Problem: Separate request creation from execution.
- ⚠️Before: Immediate execution made undoing or delaying actions difficult.
- ✅Solution: Encapsulates actions as command objects, allowing delayed execution

In [47]:
class Command:
    def execute(self):
        raise NotImplementedError

class PrintCommand(Command):
    def __init__(self, message):
        self.message = message

    def execute(self):
        print(self.message)

class CommandQueue:
    def __init__(self):
        self.queue = []

    def add_command(self, command):
        self.queue.append(command)

    def execute_all(self):
        for command in self.queue:
            command.execute()
        self.queue.clear()

queue = CommandQueue()
queue.add_command(PrintCommand("Hello"))
queue.add_command(PrintCommand("World"))
queue.execute_all()

Hello
World


## 13. State Pattern (Merge Request States):
- ❌Problem: Manage state-specific behavior dynamically.
- ⚠️Before: if-else chains made state transitions complex.
- ✅Solution: Encapsulates each state in separate classes, reducing complexity

In [48]:
class InvalidTransitionError(Exception):
    pass

class MergeRequestState:
    def open(self):
        raise NotImplementedError

    def close(self):
        raise NotImplementedError

    def merge(self):
        raise NotImplementedError

class OpenState(MergeRequestState):
    def open(self):
        print("Already open")

    def close(self, merge_request):
        print("Closing merge request")
        merge_request.state = ClosedState()

    def merge(self, merge_request):
        print("Merging and closing merge request")
        merge_request.state = MergedState()

class ClosedState(MergeRequestState):
    def open(self, merge_request):
        print("Reopening merge request")
        merge_request.state = OpenState()

    def close(self):
        print("Already closed")

    def merge(self):
        raise InvalidTransitionError("Cannot merge a closed request")

class MergedState(MergeRequestState):
    def open(self):
        raise InvalidTransitionError("Already merged")

    def close(self):
        raise InvalidTransitionError("Already merged")

    def merge(self):
        print("Already merged")

class MergeRequest:
    def __init__(self):
        self.state = OpenState()

    def open(self):
        self.state.open(self)

    def close(self):
        self.state.close(self)

    def merge(self):
        self.state.merge(self)

mr = MergeRequest()
mr.close()  # Closing merge request
mr.open()   # Reopening merge request
mr.merge()  # Merging and closing merge request

Closing merge request
Reopening merge request
Merging and closing merge request


## 14. Null Object Pattern:
- ❌Problem: Prevent None from causing runtime errors.
- ⚠️Before: AttributeError occurred when accessing None attributes.
- ✅Solution: A "do-nothing" object that behaves like a real object

In [50]:
class User:
    def __init__(self, name):
        self.name = name

    def send_message(self, message):
        print(f"Sending message to {self.name}: {message}")

class NullUser(User):
    def __init__(self):
        super().__init__("Unknown")

    def send_message(self, message):
        print("No user to send message")
def get_user(user_id):
    return User("John") if user_id == 1 else NullUser()

user = get_user(0)
user.send_message("Hello!") 

No user to send message


## 15. SharedAllMixin:
- ❌Problem: Reducing duplication in Borg-based implementations.
- ✅Solution: A Mixin ensures all attributes are shared without repetition.

In [43]:
class SharedAllMixin:
    def __init__(self, *args, **kwargs):
        try:
            self.__class__._attributes
        except AttributeError:
            self.__class__._attributes = {}

        self.__dict__ = self.__class__._attributes
        super().__init__(*args, **kwargs)

class BaseFetcher:
    def __init__(self, source):
        self.source = source

class TagFetcher(SharedAllMixin, BaseFetcher):
    def pull(self):
        return f"Pulling from tag {self.source}"

class BranchFetcher(SharedAllMixin, BaseFetcher):
    def pull(self):
        return f"Pulling from branch {self.source}"

In [44]:
tag1 = TagFetcher("v1.0")
branch1 = BranchFetcher("develop")

tag1.source = "v2.0"
print(branch1.source) 

develop


## 16. UserSource (Adapter Pattern):
- ❌Problem: Adapting external APIs to match internal interfaces.
- ⚠️Before: Direct usage of external APIs required modifications.
- ✅Solution: A wrapper class to adapt method calls​

In [51]:
class UserSource:
    def __init__(self, adaptee):
        self.adaptee = adaptee

    def fetch(self, user_id, username):
        user_namespace = self._adapt_arguments(user_id, username)
        return self.adaptee.search(user_namespace)

    @staticmethod
    def _adapt_arguments(user_id, username):
        return f"{user_id}:{username}"

#### for example:

In [52]:
external = UsernameLookup()
adapter = UserSource(external)

result = adapter.fetch(42, "john_doe")
print(result) 

Searching for 42:john_doe


# ****************************************************************************************************************

### Amitis Hashemi